# Code

In [ ]:
import pandas as pd
from pathlib import Path
import re, glob
from datetime import datetime

In [ ]:
folder_path = Path(r'./data/')
file = list(folder_path.rglob('*Fleetpin*.csv'))
df = pd.read_csv(file[0])
selector = ['Name', 'Odo / Hours / Hubo', 'Next Service Due', 'RUC Due', 'WoF Due', 'Rego Due']
df = df[selector]

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
# convert 'WoF Due' and 'Rego Due' data from string to datetime
df['WoF Due'] = pd.to_datetime(df['WoF Due'], dayfirst=True)
df['Rego Due'] = pd.to_datetime(df['Rego Due'], dayfirst=True)

# get today's date for comparison
today = datetime.now().date()

# date calculations - subtract 'WoF Due' date from today's date to get days remaining for WOF and Rego renewals
df['WOF Renew'] = (df['WoF Due'] - pd.Timestamp(today)).dt.days
df['Rego Renew'] = (df['Rego Due'] - pd.Timestamp(today)).dt.days

In [ ]:
# cleaning data by removing characters from numbers using regex
# df['Odo / Hours / Hubo'] = pd.Series(df['Odo / Hours / Hubo']).str.findall(r'\d+')
df['Odo / Hours / Hubo'] = (df['Odo / Hours / Hubo']).str.findall(r'\d+')

# get rid of list-like structure and to strings where the elements are joined by comma
df['Odo / Hours / Hubo'] = [','.join(map(str, l)) for l in df['Odo / Hours / Hubo']]

# split values seprated by comma
df['ODO'] = df['Odo / Hours / Hubo'].apply(lambda x: x.split(',')[0])
df['Hubometer'] = df['Odo / Hours / Hubo'].apply(lambda x: x.split(',')[-1])

# get rid of 'km' string at the end
df['Next Service Due'] = (df['Next Service Due']).str.strip('km')

# use regex to fetch only the numbers
df['RUC Due'] = pd.Series(df['RUC Due']).str.findall(r'\b\d+\b')
# get rid of list-like structure and to strings where the elements are joined by comma
df['RUC Due']  = [','.join(map(str, l)) for l in df['RUC Due'] ]

# convert to integer
df['Next Service Due'] = df['Next Service Due'].apply(lambda x: int(x))
df['RUC Due'] = df['RUC Due'].apply(lambda x: int(x))
df['ODO'] = df['ODO'].apply(lambda x: int(x))
df['Hubometer'] = df['Hubometer'].apply(lambda x: int(x))

# calculations
df['RUC Remaining'] = df['RUC Due'] - df['Hubometer']
df['Next Service In'] = df['Next Service Due'] - df['ODO']


In [ ]:
# change date format
df['WoF Due*'] = df['WoF Due'].apply(lambda x: x.strftime('%d-%m-%Y'))
df['Rego Due*'] = df['Rego Due'].apply(lambda x: x.strftime('%d-%m-%Y'))

In [ ]:
# re-ordering column
df = df[['Name','ODO','Hubometer','Next Service Due','RUC Due','Rego Due*','WoF Due*','Next Service In','WOF Renew','Rego Renew','RUC Remaining']]


In [ ]:
df.head()

In [ ]:
df.dtypes

# Styling

In [ ]:
df_styled = df.style\
    .applymap(lambda x: 'background-color: %s' % '#ff9966' if x < 0 else ('background-color: %s' % '#99ff99' if 0 <= x <= 5000 else 'background-color: %s' % 'none'), subset=['Next Service In'])\
    .applymap(lambda x: 'background-color: %s' % '#ff9966' if x < 0 else 'background-color: %s' % 'none', subset=['RUC Remaining'])\
    .highlight_between(subset='RUC Remaining',left=0,right=7000, color='#ff9966')\
    .applymap(lambda x: 'background-color: %s' % '#ff9966' if x < 0 else ('background-color: %s' % '#99ff99' if 0 <= x <= 30 else 'background-color: %s' % 'none'), subset=['WOF Renew'])\
    .applymap(lambda x: 'background-color: %s' % '#ff9966' if x < 0 else ('background-color: %s' % '#99ff99' if 0 <= x <= 30 else 'background-color: %s' % 'none'), subset=['Rego Renew'])\
    .format({'RUC Remaining': '{0:,.0f} km', 'Next Service In': '{0:,.0f} km', 'ODO': '{0:,.0f} km', 'Hubometer': '{0:,.0f} km', 'Next Service Due': '{0:,.0f} km', 'RUC Due': '{0:,.0f} km',
             'WOF Renew': '{} Days', 'Rego Renew': '{} Days'})



# Export to excel

In [ ]:
folder_path_report = Path(r'./report/')
file_path = folder_path_report / 'Report.xlsx'
df_styled.to_excel(file_path, engine='openpyxl', index=False)

# Report

In [ ]:
df_styled